In [1]:
import numpy as np
from tensorflow import keras
import os
import cv2
from os import listdir
from os.path import isfile
from os.path import join
import time

#######################################################################################
traindata_path = r"C:\Users\foo\Desktop\mnist\Training data1"

numbers_of_traindata = 0

base_x_train = []
base_y_train = []

label_folder = []

#################################Read Train Data########################################
for root, dirts, files in os.walk(traindata_path):
    for dirt in dirts:
        label_folder.append(dirt)
    numbers_of_traindata += len(files)
    
for i in range(len(label_folder)):
    label_path = traindata_path + r"\\" + label_folder[i]
    trainfileName = [f for f in listdir(label_path) if isfile(join(label_path, f))]
    
    for j in range(len(trainfileName)):
        path2 = label_path + r"\\" + trainfileName[j]
        img = cv2.imread(path2, cv2.IMREAD_GRAYSCALE)
        base_x_train.append(img)
        base_y_train.append(label_folder[i])

real_x_train = np.array(base_x_train).reshape(np.array(base_x_train).shape[0], 784)
real_y_train = np.array(base_y_train).reshape(np.array(base_y_train).shape[0], 1)

########################################################################################

In [2]:
import numpy as np
from tensorflow import keras
import os
import cv2
from os import listdir
from os.path import isfile
from os.path import join
import time

class NN:
    def __init__(self, sizes, lr = 0.7):
        self.sizes = sizes
        self.lr = lr
        self.parameter = self.Init_parameter()
        
    def Init_parameter(self):   
        input_layer = self.sizes[0]
        hidden_layer_1 = self.sizes[1]
        hidden_layer_2 = self.sizes[2]
        output_layer = self.sizes[3]

        parameter = {
            'w1':np.random.normal(0., 1.,(hidden_layer_1, input_layer)) * np.sqrt(1. / hidden_layer_1) ,
            'w2':np.random.normal(0., 1.,(hidden_layer_2, hidden_layer_1)) * np.sqrt(1. / hidden_layer_2) ,
            'w3':np.random.normal(0., 1.,(output_layer, hidden_layer_2)) * np.sqrt(1. / output_layer) ,
        }
        return parameter
    
    def Forward_pass(self, x):
        parameter = self.parameter       
        parameter['a0'] = x
        #print(x)

        # hidden layer 1
        parameter['s1'] = np.dot(parameter["w1"], parameter['a0'])
        parameter['a1'] = self.Sigmoid(parameter['s1'])

        # hidden layer 2
        parameter['s2'] = np.dot(parameter["w2"], parameter['a1'])
        parameter['a2'] = self.Sigmoid(parameter['s2'])

        # output layer
        parameter['s3'] = np.dot(parameter["w3"], parameter['a2'])
        parameter['a3'] = self.Sigmoid(parameter['s3'])

        return parameter['a3']
    
    def Backward_pass(self, y, output):
        parameter = self.parameter
        changed_w = {}
        
        # update w
        error =  (output - y) * self.Sigmoid(parameter['s3'], derivative = True)
        changed_w['w3'] = np.outer(error, parameter['a2'])
        
        error = np.dot(parameter['w3'].T, error) * self.Sigmoid(parameter['s2'], derivative = True)
        changed_w['w2'] = np.outer(error, parameter['a1'])
        
        error = np.dot(parameter['w2'].T, error) * self.Sigmoid(parameter['s1'], derivative = True)
        changed_w['w1'] = np.outer(error, parameter['a0'])

        return changed_w

    def Update(self, changed_w):
        for key, value in changed_w.items():
            delW = self.lr * value
            self.parameter[key] -= delW

    def Sigmoid(self, z, derivative = False):
        if derivative:
            return (np.exp(-z))/((np.exp(-z)+1)**2)
        return 1/(1 + np.exp(-z))
    
    #def Softmax(self, z, derivative = False):
        #exps = np.exp(z - z.max())
        #if derivative:
            #return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))
        #return exps / np.sum(exps, axis=0)
    
    def Train(self, x_train, y_train, x_test, y_test):
        for x, y in zip(x_train, y_train):
            x = x.reshape(784, 1)
            y = y.reshape(10, 1)
            f_result = self.Forward_pass(x)
            b_result = self.Backward_pass(y, f_result)
            self.Update(b_result)
        self.lr *= 0.95
        ###########################Validate###########################
        error = 0
        for x, y in zip(x_test, y_test):
            x = x.reshape(784, 1)
            y = y.reshape(10, 1)
            f_result = self.Forward_pass(x)
            if np.argmax(f_result) != np.argmax(y):
                error += 1
        print('Acc = ', 1 - error/ len(x_test), ' lr = ', self.lr)
        ##############################################################
    def Predict(self, real_x_test):
        filecounter = 0
        with open(r"C:\Users\foo\Desktop\mnist\410785018.txt", 'w') as f:
            for x in real_x_test:
                x = x.reshape(784, 1)
                f_result = self.Forward_pass(x)
                txt_name = os.path.splitext(testfileName[filecounter].split('.')[0])[0]
                f.write(txt_name + " " + (np.argmax(f_result)).astype(str) + "\n")
                filecounter += 1

In [3]:
if __name__ == '__main__':
    real_x_train = real_x_train.astype('float32')
    real_x_train /= 255
   
    real_y_train = keras.utils.to_categorical(real_y_train,num_classes = 10)
  
    model = NN(sizes=[784, 128, 64, 10])
    epoch = 20
    for i in range(epoch):
        print("Epoch = ", i)
        p = np.random.permutation(len(real_x_train))
        real_x_train = real_x_train[p]
        real_y_train = real_y_train[p]
        model.Train(real_x_train, real_y_train, real_x_train, real_y_train)

Epoch =  0
Acc =  0.9723  lr =  0.6649999999999999
Epoch =  1
Acc =  0.98355  lr =  0.6317499999999999
Epoch =  2
Acc =  0.9865  lr =  0.6001624999999999
Epoch =  3
Acc =  0.99045  lr =  0.5701543749999999
Epoch =  4
Acc =  0.9923  lr =  0.5416466562499999
Epoch =  5
Acc =  0.99615  lr =  0.5145643234374998
Epoch =  6
Acc =  0.9957  lr =  0.48883610726562476
Epoch =  7
Acc =  0.9975  lr =  0.4643943019023435
Epoch =  8
Acc =  0.9982  lr =  0.44117458680722627
Epoch =  9
Acc =  0.99855  lr =  0.4191158574668649
Epoch =  10
Acc =  0.99865  lr =  0.39816006459352166
Epoch =  11
Acc =  0.9987  lr =  0.37825206136384554
Epoch =  12
Acc =  0.99885  lr =  0.3593394582956532
Epoch =  13
Acc =  0.99885  lr =  0.34137248538087056
Epoch =  14
Acc =  0.99885  lr =  0.324303861111827
Epoch =  15
Acc =  0.99885  lr =  0.30808866805623564
Epoch =  16
Acc =  0.99885  lr =  0.29268423465342386
Epoch =  17
Acc =  0.99885  lr =  0.27805002292075265
Epoch =  18
Acc =  0.9989  lr =  0.264147521774715
Epoch

In [4]:
testdata_path = r"C:\Users\foo\Desktop\mnist\Testing data1"
numbers_of_testdata = 0
base_x_test = []
####################################Read Test Data########################################################

for dirs, subdirs, files in os.walk(testdata_path):
    numbers_of_testdata += len(files)

testfileName = [file for file in listdir(testdata_path) if isfile(join(testdata_path, file))]

for num, fname in enumerate(testfileName):
    path = testdata_path + r"\\" + fname
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)        
    base_x_test.append(img)

real_x_test = np.array(base_x_test).reshape(np.array(base_x_test).shape[0], 784)
##########################################################################################################
real_x_test = real_x_test.astype('float32')
real_x_test /= 255
model.Predict(real_x_test)